<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/1_Finans_i%C3%A7in_Python_SMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kütüphanelerin yüklenmesi
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener
!pip install backtesting

#kütüphanelerin çağırılması
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy

#Tradingview Kütüphanesini açıyoruz.
tv = TvDatafeed()

#Hisse Adını Tanımlıyoruz.
Hisse = "EREGL"

#Hisse Verilerini Tradingview Kütüphanesinden çekiyoruz.
data = tv.get_hist(symbol=Hisse, exchange='BIST', interval=Interval.in_1_hour, n_bars=1000)
print(data)

#SMA hesaplamasını tanımlıyoruz.
def sma(series, length):
    calc = series.rolling(window=length).mean()
    return calc

#SMA 5 i hesaplatıyoruz.
data['SMA5'] = sma(data['close'],5)
print(data)

#SMA 20 i hesaplatıyoruz.
data['SMA20'] = sma(data['close'],20)
print(data)

#SMA 5 SMA20 nin yukarısında ise giriş koşulu
data['Entry'] = data['SMA5'] > data['SMA20']

#SMA 5 SMA20 nin yukarısında ise giriş koşulu
data['Exit'] = data['SMA5'] < data['SMA20']
print(data)

#Her Zaman kullanacağımız Stratejimizi hazırlıyoruz.
class Strategy(Strategy):
    def init(self):
        pass
    def next(self):
        if self.data['Entry'] == True and not self.position:
            self.buy()
        elif self.data['Exit'] == True:
            self.position.close()

#data başlıklarını yeniden isimlendiriyoruz.
data.rename(columns={'open': 'Open', 'high': 'High',
                     'low': 'Low',
                     'close': 'Close',
                     'volume': 'Volume'},
                     inplace=True)

#Backtest yapımızı çağırıyoruz ve sonuçları inceliyoruz.
bt = Backtest(data, Strategy, cash=100000, commission=0.002)
Stats = bt.run()
print(Stats)

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-kfjk4qg2
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-kfjk4qg2
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


                         symbol   open   high    low  close       volume
datetime                                                                
2024-02-19 10:00:00  BIST:EREGL  48.92  49.14  48.82  48.94    8653462.0
2024-02-19 11:00:00  BIST:EREGL  48.96  48.96  48.74  48.80    7409330.0
2024-02-19 12:00:00  BIST:EREGL  48.78  48.86  48.66  48.76    7497085.0
2024-02-19 13:00:00  BIST:EREGL  48.76  48.78  48.32  48.52   18201000.0
2024-02-19 14:00:00  BIST:EREGL  48.52  48.86  47.80  48.44   42600864.0
...                         ...    ...    ...    ...    ...          ...
2024-07-19 11:00:00  BIST:EREGL  56.50  56.60  54.85  55.00   39899671.0
2024-07-19 12:00:00  BIST:EREGL  55.00  58.20  55.00  57.35  120674384.0
2024-07-19 13:00:00  BIST:EREGL  57.40  58.10  57.10  57.55   58274378.0
2024-07-19 14:00:00  BIST:EREGL  57.55  58.15  57.30  57.50   43382637.0
2024-07-19 15:00:00  BIST:EREGL  57.50  57.55  57.50  57.55   13398135.0

[1000 rows x 6 columns]
                         s